# New version

In [49]:
#----------------------------------------------------------------------------------
#Author: Veerle Martens
#Date: may 2012
#Adapted by: Raquel Ubach (UAB) - Dec 2023/Jul 2024
#
#Purpose: Create service areas around each building block and find out the area of
#         green urban area that can be reached.
#---------------------------------------------------------------------------------

#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(input_layer, output_layer, network_path, group_id, nb_meters):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.MakeServiceAreaLayer_na(network_path, "{}_SA_400m".format(group_id), "Meters", "TRAVEL_FROM", nb_meters, "DETAILED_POLYS", "NO_MERGE", "DISKS", "NO_LINES", "", "", "", "", "ALLOW_UTURNS", "", "NO_TRIM_POLYS", "", "")
    arcpy.AddLocations_na(salyr, "Facilities", input_layer, "Name {} #".format(UATL_ID_Fld), "500 Meters", "", "", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "", "EXCLUDE", "")
    print("Solving network analysis")
    arcpy.Solve_na(salyr, "SKIP")
    return salyr

In [5]:
# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step1')
temp_f = os.path.join(indata_f, 'Processing', 'step2_CleaningGQA')
step2_f = os.path.join(indata_f, 'OutputData', 'batch1_allprocessed', 'step2')
if not os.path.exists(step2_f):
    os.makedirs(step2_f)


In [13]:
# Create list of urban centres
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2.shp')

# Specify the columns you need to extract and apply conditions on
columns = ["Batch", "CNTR_CODE", "HDENS_CLST"]

# Initialize an empty list to store the values
values_list = []

# Use an arcpy.da.SearchCursor to iterate over the rows in the shapefile
with arcpy.da.SearchCursor(uc_file_path, columns) as cursor:
    for row in cursor:
        # Apply the selection criteria
        if row[0] == 1.0:
            values_list.append(row[2])

In [12]:
# input parameters
pedestrian_path = networks_f
outPath = step2_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"
nbMeters = 400

In [11]:
# Define the path and name for the geodatabase
gdb_path = temp_f
gdb_name = "SA_output_batch1_v3007.gdb"
outGDB = os.path.join(gdb_path, gdb_name)

# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

In [16]:
# Create a list of files in the directory that end with GQA.shp
GQA_ls = [
    os.path.join(directory, f) for f in os.listdir(directory)
    if f.endswith('finalGQA.shp')
]

In [26]:
import pandas as pd
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

In [27]:
codes.head()

,FREQUENCY,HDENS_CLST,HDENS_NAME,HDENS_2011,agglomerationId_identifier,agglomerationName_nameEng,UA2018
0,1,GEOSTAT21_002,Oulu / Uleåborg,GEOSTAT11_002,AG_FI_00_6,Oulu,FI004L4_OULU_UA2018_v013
1,1,GEOSTAT21_003,Umeå,GEOSTAT11_003,SE_a_ag2480,Umea,SE005L1_UMEA_UA2018_v013
2,1,GEOSTAT21_004,Trondheim,GEOSTAT11_004,AG_NO_00_3,Trondheim,NO003L1_TRONDHEIM_UA2018_v013
3,1,GEOSTAT21_005,Jyväskylä,GEOSTAT11_005,AG_FI_00_9,Jyvaskyla,FI009L2_JYVASKYLA_UA2018_v013
4,1,GEOSTAT21_006,Tampere / Tammerfors,GEOSTAT11_006,AG_FI_00_3,Tampere,FI002L3_TAMPERE_UA2018_v013


In [ ]:
HDENS_AGGL_city_aglid = HDENS_AGGL_tbl.query(f"HDENS_CLST=='{uc_city_code}' & agglomerationId_identifier=='{agl_id}'")
ctry_code = HDENS_AGGL_city_aglid.CNTR_CODE.values.astype(str)[0]

In [32]:
fields = arcpy.ListFields(uc_file_path)
for field in fields:
    print(f"Field name: {field.name}")

Field name: FID
Field name: Shape
Field name: HDENS_CLST
Field name: HDENS_NAME
Field name: HDENS_2011
Field name: POPL_2021
Field name: CNTR_CODE
Field name: MBRS_CODE_
Field name: SHAPE_AREA
Field name: SHAPE_LEN
Field name: Batch


In [38]:
GQA_ls[0]

'P:\\Environment and Health\\Noise\\ServiceContract\\2024_ServiceContract\\QuietAreas\\Processing\\step2_CleaningGQA\\GEOSTAT21_021_finalGQA.shp'

In [48]:
arcpy.CheckInExtension("Network")
if arcpy.CheckExtension("Network") == "Available":
    arcpy.CheckOutExtension("Network")
else:
    raise CustomError("The Network Analyst extension is not available.")

#Environment settings
arcpy.env.overwriteOutput = 'true'
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2.shp')
sql_query = "Batch = 1"
with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        geom = uc_row[0]  # Geometry object
        ctr_cd = uc_row[2]
        uc_code = uc_row[1]
        print(uc_code)
        
        inGQA = os.path.join(temp_f, '{}_finalGQA.shp'.format(uc_code))
        if os.path.exists(inGQA):
            print(inGQA)
            pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
            if not arcpy.Exists(pedestrianNWpath):
                raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

            arcpy.env.extent = geom.extent # Extent object of the geometry
            print("Extent set to match uc: {}".format(arcpy.env.extent))

            try:          

                if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                    arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                ## Count number of GUA Blocks
                cnt = arcpy.GetCount_management(uatlLyr)
                print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                ## Create and Select points from GUA borders used to create SA
                print ("Create points from GUA borders used to create SA")

                #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                if arcpy.Exists(fc_GUA_pt_50m):
                    arcpy.Delete_management(fc_GUA_pt_50m)
                print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')
                print ("Points generated - hurray!!")

                cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                nbrec = cnt.getOutput(0)
                print ("nb generated points : " + nbrec)

                if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                    arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                    arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                

                desc = arcpy.Describe(uatlLyr)
                SR = desc.spatialReference
                flds = arcpy.ListFields(uatlLyr)
                ###UATL_FieldName_List = [fld.name for fld in flds if fld.name != desc.OIDFieldName and fld.type != 'Geometry' and fld.name != desc.areaFieldName and fld.name != desc.lengthFieldName]
                UATL_FieldName_List = [fld.name for fld in flds if fld.type != 'Geometry']
                # rename FID to OBJECTID
                UATL_FieldName_List = ['OBJECTID' if field == 'FID' else field for field in UATL_FieldName_List]


                ##fc_GUA_centroids_no_pt = outPath + "\\GUA_centroids_no_pt.shp"
                fc_GUA_centroids_no_pt = os.path.join(outGDB, '{}GUA_centroids_no_pt'.format(uc_code))    
            ##    fc_GUA_centroids_no_pt = r"in_memory\GUA_centroids_no_pt"
                if arcpy.Exists(fc_GUA_centroids_no_pt):
                    arcpy.Delete_management(fc_GUA_centroids_no_pt)
                print ("CreateFeatureclass " + os.path.basename(fc_GUA_centroids_no_pt))
                ##arcpy.CreateFeatureclass_management("in_memory", "GUA_centroids_no_pt", "POINT", "", "DISABLED", "DISABLED", SR, "", "0", "0", "0")
                arcpy.CreateFeatureclass_management(outGDB, "{}GUA_centroids_no_pt".format(uc_code), "POINT", "", "DISABLED", "DISABLED", SR, "", "0", "0", "0")
                
                idxfld = 0
                for fldnam in UATL_FieldName_List:
                    for fldidx in range(0,len(flds)):
                        if fldnam == flds[fldidx].name:
                            typeF = flds[fldidx].type
                            if typeF == 'String':
                                length_fld = flds[fldidx].length
                            break
                    if typeF in ['Integer']:
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='LONG')
                    elif typeF == 'SmallInteger':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='SHORT')
                    elif typeF == 'String':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='TEXT',field_length=length_fld)
                    elif typeF == 'Double':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='DOUBLE')
                    elif typeF == 'Single':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='FLOAT')
                    elif typeF == 'Date':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='DATE')
                    idxfld += 1

                fields = []
                fields.append("SHAPE@")
                for fldnam in UATL_FieldName_List:
                    fields.append(fldnam)
                print ("Insert centroids where no point in " + os.path.basename('{}fc_GUA_pt_50m_nw_25m').format(uc_code))
                cursor = arcpy.da.InsertCursor(fc_GUA_centroids_no_pt, fields)  
                for row in arcpy.da.SearchCursor(uatlLyr, fields):
                    if row[1] not in UATL_Ids_GUAs_nw_25m:
                        list_row = list(row)
                        list_row[0] = row[0].centroid
                        cursor.insertRow(convert(list_row))  
                del row,cursor

                print("centroids inserted")

                ##fc_GUA_mem_pt_all = "in_memory\GUA_pt_ALL"
                fc_GUA_mem_pt_all = outGDB +"\\{}_GUA_pt_ALL".format(uc_code)
                ###fc_GUA_mem_pt_all = outPath + "\\\GUA_pt_ALL.shp"

                if arcpy.Exists(fc_GUA_mem_pt_all):
                    arcpy.Delete_management(fc_GUA_mem_pt_all)
                print ("CopyFeatures from " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " in "  + fc_GUA_mem_pt_all)
                arcpy.CopyFeatures_management(fc_GUA_pt_50m_nw_25m, fc_GUA_mem_pt_all) 
                print ("append " + os.path.basename(fc_GUA_centroids_no_pt) + " in " + os.path.basename(fc_GUA_mem_pt_all))
                arcpy.Append_management(fc_GUA_centroids_no_pt, fc_GUA_mem_pt_all, "NO_TEST")

                cnt = arcpy.GetCount_management(fc_GUA_mem_pt_all)
                nbrec = cnt.getOutput(0)
                print ("nb points in " + os.path.basename(fc_GUA_mem_pt_all) + " : " + nbrec)

                
                #Loop through to prevent memory errors
                # Batch processing to prevent memory issues
                batch_size = 1000 
                print('batch size 1000')

                # Function to create service area layer
                for i in range(0, int(nbrec), batch_size):
                    j = min(i + batch_size, int(nbrec))
                    print ("this is i {} and this is j {}".format(i,j))
                    expr = "\"OBJECTID\" >= {} AND \"OBJECTID\" < {}".format(i,j)

                    if arcpy.Exists("GUA_points_layer"):
                        arcpy.Delete_management("GUA_points_layer")
                    GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_mem_pt_all, "{}GUA_points_layer".format(uc_code), expr)
                    print('GUA_pt_lyr')

                    # Create service area layer
                    salyr = create_service_area_layer(GUA_pt_lyr, "{}_SA_400m_batch_{}_{}".format(uc_code,i,j), pedestrianNWpath, ctr_cd, nbMeters)

                    if i == 0:
                        fc_SA = os.path.join(gdb_path, gdb_name, "{}_sa".format(uc_code))
                        if arcpy.Exists(fc_SA):
                            arcpy.Delete_management(fc_SA)
                        #saFC = arcpy.CopyFeatures_management(salyr + "\\Polygons", fc_SA)
                        saFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
                        
                    else:
                        tmpFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), os.path.join(gdb_path, gdb_name, "{}_sa_batch_{}_{}").format(ctr_cd,i,j))
                        arcpy.Append_management(tmpFC, saFC, "NO_TEST")
                        arcpy.Delete_management(tmpFC)

                    arcpy.Delete_management(salyr) 

                fc_GUA_pt_all = os.path.join(gdb_path, gdb_name, "GUA_{}_pt_ALL".format(uc_code))
                if arcpy.Exists(fc_GUA_pt_all):
                    arcpy.Delete_management(fc_GUA_pt_all)
                print ("CopyFeatures from " + os.path.basename(fc_GUA_mem_pt_all) + " in "  + fc_GUA_pt_all)
                arcpy.CopyFeatures_management(fc_GUA_mem_pt_all, fc_GUA_pt_all)
                

                print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                
            except CustomError as ce:
                arcpy.AddError(ce.value)
                print (ce.value)
            except arcpy.ExecuteError:
                msgs = arcpy.GetMessages(2)
                arcpy.AddError(msgs)
                print (msgs)
            except:
                tb = sys.exc_info()[2]
                tbinfo = traceback.format_tb(tb)[0]
                pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                arcpy.AddError(pymsg)
                print (msgs)
                arcpy.AddError(msgs)
                print (pymsg)
            finally:
                arcpy.CheckInExtension("Network")




SE GEOSTAT21_018 4763000 4038000 4791000 4065000 NaN NaN NaN NaN
P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\step2_CleaningGQA\GEOSTAT21_018_finalGQA.shp
The Network Analyst extension is not available.
EE GEOSTAT21_021 5288000 4030000 5294000 4036000 NaN NaN NaN NaN
P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\step2_CleaningGQA\GEOSTAT21_021_finalGQA.shp
The Network Analyst extension is not available.
NO GEOSTAT21_023 4071000 3976000 4079000 3998000 NaN NaN NaN NaN
P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\step2_CleaningGQA\GEOSTAT21_023_finalGQA.shp
The Network Analyst extension is not available.
SE GEOSTAT21_024 4677000 3955000 4685000 3961000 NaN NaN NaN NaN
P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\step2_CleaningGQA\GEOSTAT21_024_finalGQA.shp
The Network Analyst extension is not available.
SE G

In [ ]:
arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')

In [ ]:
import arcpy

# Define the path to the shapefile
shapefile_path = r"C:\path\to\your\shapefile.shp"

# Initialize variables to store the combined extent
xmin, ymin, xmax, ymax = None, None, None, None

# Check if the shapefile exists
if arcpy.Exists(shapefile_path):
    # Open a search cursor to iterate through the features
    with arcpy.da.SearchCursor(shapefile_path, ["SHAPE@"]) as cursor:
        for row in cursor:
            geom = row[0]  # Geometry object
            extent = geom.extent  # Extent object of the geometry
            
            # Update the combined extent
            if xmin is None or extent.XMin < xmin:
                xmin = extent.XMin
            if ymin is None or extent.YMin < ymin:
                ymin = extent.YMin
            if xmax is None or extent.XMax > xmax:
                xmax = extent.XMax
            if ymax is None or extent.YMax > ymax:
                ymax = extent.YMax
    
    # Set the environment extent using the combined extent
    if xmin is not None and ymin is not None and xmax is not None and ymax is not None:
        arcpy.env.extent = arcpy.Extent(xmin, ymin, xmax, ymax)
        print(f"Environment extent set to: {arcpy.env.extent}")
    else:
        print("No valid features found in the shapefile to set extent.")
else:
    print(f"Shapefile does not exist at {shapefile_path}")


In [28]:
# 1 COMMON SOURCES FOR ALL DATA
# URBAN CENTRES
# Read shapefile
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB1B2.shp')
# Read the GeoPackage file
uc = gpd.read_file(uc_file_path)
uc['CNTR_CODE'].fillna('AA', inplace=True)

# Select cities for processing in this batch
uc_sel = uc.query('Batch==1.0')
uc_sel = uc_sel.sort_values(by='CNTR_CODE')

# Read table to list the cities to process using urban centre code
cities_ls = uc_sel.HDENS_CLST.tolist()

# NOISE DATA
# Load agglomerations delineations
agls_file_path = os.path.join(indata_f, 'NoiseData', 'DF1_5_Agglomerations_20240429.gpkg')

# Read the GeoPackage file
agls = gpd.read_file(agls_file_path, layer = 'dbo.DF15_AgglomerationSource_Valid_LatestDelivery', 
                     **engines['pyogrio+arrow'],columns=['agglomerationId_identifier', 'agglomerationName_nameEng', 'geometry'])

# URBAN ATLAS
# Read table with HDENS Urban centres information and Agglomerations link
HDENS_AGGL_tbl = pd.read_csv(r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\UrbanCentres_Agglomerations_csv.csv')
# Join uc code field to this table
HDENS_AGGL_tbl = HDENS_AGGL_tbl.merge(uc[['POPL_2021', 'HDENS_CLST']], on='POPL_2021')
# UA data folder
ua_data_f = r'A:\Copernicus\UrbanAtlas\UrbanAtlas\UA2018'
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

NameError: name 'gpd' is not defined

In [ ]:
for i in GQA_ls:
    uc_code = os.path.basename(GQA_ls[0])[:-13]
    print(uc_code)
    codes.query(f"HDENS_CLST=='{uc_code}'")

In [ ]:
GEOSTAT21_002_finalGQA.shp

In [ ]:
outCentroids = os.path.join(temp_f, f'{uc_code}_pts_GQA.shp')

In [ ]:
arcpy.CheckInExtension("Network")
if arcpy.CheckExtension("Network") == "Available":
    arcpy.CheckOutExtension("Network")
else:
    raise CustomError("The Network Analyst extension is not available.")

#Environment settings
arcpy.env.overwriteOutput = 'true'
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2.shp')
sql_query = "Batch = 1"
with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        geom = uc_row[0]  # Geometry object
        ctr_cd = uc_row[2]
        uc_code = uc_row[1]
        print(uc_code)
        
        inGQA = os.path.join(temp_f, '{}_finalGQA.shp'.format(uc_code))
        if os.path.exists(inGQA):
            print(inGQA)
            inGQA = inGQA
            ctr_cd = ctr_cd

            pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
            if not arcpy.Exists(pedestrianNWpath):
                raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

            arcpy.env.extent = geom.extent # Extent object of the geometry
            print("Extent set to match uc: {}".format(arcpy.env.extent))

            try:          

                if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                    arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                ## Count number of GUA Blocks
                cnt = arcpy.GetCount_management(uatlLyr)
                print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                ## Create and Select points from GUA borders used to create SA
                print ("Create points from GUA borders used to create SA")

                #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                if arcpy.Exists(fc_GUA_pt_50m):
                    arcpy.Delete_management(fc_GUA_pt_50m)
                print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')
                print ("Points generated - hurray!!")

                cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                nbrec = cnt.getOutput(0)
                print ("nb generated points : " + nbrec)

                if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                    arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                    arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                print("Nb of GQA near network " +len(UATL_Ids_GUAs_nw_25m))
                ids_ls_GQA = unique_values(uatlLyr, "FID")
                print("Total nb of GQA " +len(ids_ls_GQA))

                # GQA without access to network
                ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
                

                

                desc = arcpy.Describe(uatlLyr)
                SR = desc.spatialReference
                flds = arcpy.ListFields(uatlLyr)
                ###UATL_FieldName_List = [fld.name for fld in flds if fld.name != desc.OIDFieldName and fld.type != 'Geometry' and fld.name != desc.areaFieldName and fld.name != desc.lengthFieldName]
                UATL_FieldName_List = [fld.name for fld in flds if fld.type != 'Geometry']
                # rename FID to OBJECTID
                UATL_FieldName_List = ['OBJECTID' if field == 'FID' else field for field in UATL_FieldName_List]


                ##fc_GUA_centroids_no_pt = outPath + "\\GUA_centroids_no_pt.shp"
                fc_GUA_centroids_no_pt = os.path.join(outGDB, '{}GUA_centroids_no_pt'.format(uc_code))    
            ##    fc_GUA_centroids_no_pt = r"in_memory\GUA_centroids_no_pt"
                if arcpy.Exists(fc_GUA_centroids_no_pt):
                    arcpy.Delete_management(fc_GUA_centroids_no_pt)
                print ("CreateFeatureclass " + os.path.basename(fc_GUA_centroids_no_pt))
                ##arcpy.CreateFeatureclass_management("in_memory", "GUA_centroids_no_pt", "POINT", "", "DISABLED", "DISABLED", SR, "", "0", "0", "0")
                arcpy.CreateFeatureclass_management(outGDB, "{}GUA_centroids_no_pt".format(uc_code), "POINT", "", "DISABLED", "DISABLED", SR, "", "0", "0", "0")
                
                idxfld = 0
                for fldnam in UATL_FieldName_List:
                    for fldidx in range(0,len(flds)):
                        if fldnam == flds[fldidx].name:
                            typeF = flds[fldidx].type
                            if typeF == 'String':
                                length_fld = flds[fldidx].length
                            break
                    if typeF in ['Integer']:
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='LONG')
                    elif typeF == 'SmallInteger':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='SHORT')
                    elif typeF == 'String':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='TEXT',field_length=length_fld)
                    elif typeF == 'Double':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='DOUBLE')
                    elif typeF == 'Single':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='FLOAT')
                    elif typeF == 'Date':
                        arcpy.AddField_management(fc_GUA_centroids_no_pt,fldnam,field_type='DATE')
                    idxfld += 1

                fields = []
                fields.append("SHAPE@")
                for fldnam in UATL_FieldName_List:
                    fields.append(fldnam)
                print ("Insert centroids where no point in " + os.path.basename('{}fc_GUA_pt_50m_nw_25m').format(uc_code))
                cursor = arcpy.da.InsertCursor(fc_GUA_centroids_no_pt, fields)  
                for row in arcpy.da.SearchCursor(uatlLyr, fields):
                    if row[1] not in UATL_Ids_GUAs_nw_25m:
                        list_row = list(row)
                        list_row[0] = row[0].centroid
                        cursor.insertRow(convert(list_row))  
                del row,cursor

                print("centroids inserted")

                ##fc_GUA_mem_pt_all = "in_memory\GUA_pt_ALL"
                fc_GUA_mem_pt_all = outGDB +"\\{}_GUA_pt_ALL".format(uc_code)
                ###fc_GUA_mem_pt_all = outPath + "\\\GUA_pt_ALL.shp"

                if arcpy.Exists(fc_GUA_mem_pt_all):
                    arcpy.Delete_management(fc_GUA_mem_pt_all)
                print ("CopyFeatures from " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " in "  + fc_GUA_mem_pt_all)
                arcpy.CopyFeatures_management(fc_GUA_pt_50m_nw_25m, fc_GUA_mem_pt_all) 
                print ("append " + os.path.basename(fc_GUA_centroids_no_pt) + " in " + os.path.basename(fc_GUA_mem_pt_all))
                arcpy.Append_management(fc_GUA_centroids_no_pt, fc_GUA_mem_pt_all, "NO_TEST")

                cnt = arcpy.GetCount_management(fc_GUA_mem_pt_all)
                nbrec = cnt.getOutput(0)
                print ("nb points in " + os.path.basename(fc_GUA_mem_pt_all) + " : " + nbrec)

                
                #Loop through to prevent memory errors
                # Batch processing to prevent memory issues
                batch_size = 1000 
                print('batch size 1000')

                # Function to create service area layer
                for i in range(0, int(nbrec), batch_size):
                    j = min(i + batch_size, int(nbrec))
                    print ("this is i {} and this is j {}".format(i,j))
                    expr = "\"OBJECTID\" >= {} AND \"OBJECTID\" < {}".format(i,j)

                    if arcpy.Exists("GUA_points_layer"):
                        arcpy.Delete_management("GUA_points_layer")
                    GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_mem_pt_all, "{}GUA_points_layer".format(uc_code), expr)
                    print('GUA_pt_lyr')

                    # Create service area layer
                    salyr = create_service_area_layer(GUA_pt_lyr, "{}_SA_400m_batch_{}_{}".format(uc_code,i,j), pedestrianNWpath, ctr_cd, nbMeters)

                    if i == 0:
                        fc_SA = os.path.join(gdb_path, gdb_name, "{}_sa".format(uc_code))
                        if arcpy.Exists(fc_SA):
                            arcpy.Delete_management(fc_SA)
                        #saFC = arcpy.CopyFeatures_management(salyr + "\\Polygons", fc_SA)
                        saFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
                        
                    else:
                        tmpFC = arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), os.path.join(gdb_path, gdb_name, "{}_sa_batch_{}_{}").format(ctr_cd,i,j))
                        arcpy.Append_management(tmpFC, saFC, "NO_TEST")
                        arcpy.Delete_management(tmpFC)

                    arcpy.Delete_management(salyr) 

                fc_GUA_pt_all = os.path.join(gdb_path, gdb_name, "GUA_{}_pt_ALL".format(uc_code))
                if arcpy.Exists(fc_GUA_pt_all):
                    arcpy.Delete_management(fc_GUA_pt_all)
                print ("CopyFeatures from " + os.path.basename(fc_GUA_mem_pt_all) + " in "  + fc_GUA_pt_all)
                arcpy.CopyFeatures_management(fc_GUA_mem_pt_all, fc_GUA_pt_all)
                

                print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                
            except CustomError as ce:
                arcpy.AddError(ce.value)
                print (ce.value)
            except arcpy.ExecuteError:
                msgs = arcpy.GetMessages(2)
                arcpy.AddError(msgs)
                print (msgs)
            except:
                tb = sys.exc_info()[2]
                tbinfo = traceback.format_tb(tb)[0]
                pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                arcpy.AddError(pymsg)
                print (msgs)
                arcpy.AddError(msgs)
                print (pymsg)
            finally:
                arcpy.CheckInExtension("Network")


